# RSI strategy
This is an attempt to bactest the 10 biggest crypto coins by market capitalization (excluding stable coins) using the RSI indicator. The RSI is a widely used technical indicator and an oscillator that indicates a market is overbought when the RSI value is over 70 and indicates oversold conditions when RSI readings are under 30.

The coins we would be testing are BTC - Bitcoin, Ethereum, Chain Native Token, Ripple, Enegra, Cardano, Solana, DogeCoin, Polygon and Dai. 

Let's start with Bitcoin

## Bitcoin

In [133]:
import pandas as pd
import vectorbt as vbt
import yfinance as yf

def rsi_backtest(coin):
    coin_price = vbt.YFData.download(coin, missing_index='drop').get("Close")
    coin_rsi = vbt.RSI.run(coin_price)
    entries = coin_rsi.rsi_crossed_below(30)
    exits = coin_rsi.rsi_crossed_above(70)

    # set up portfolio
    pf = vbt.Portfolio.from_signals(coin_price, entries, exits)
    ret = pf.total_return()
    sr = pf.sharpe_ratio()
    max_drawdown = pf.max_drawdown()
    num_trades = pf.stats()[11]
    percent_avg_win_trades = pf.stats()[18]
    percent_avg_loss_trades = pf.stats()[19]
    metrics = {"Return": ret, 
               "Number of trades": num_trades,
               "Winning trades (%)": percent_avg_win_trades,
               "Losing trades (%)": percent_avg_loss_trades,
              "Max drawdown": max_drawdown, "Sharpe ratio": sr}
    return metrics
    
def plot_pf(met):
    for key,value in met.items():
        print(key, " => ", value)
    
btc = rsi_backtest("BTC-USD")
plot_pf(btc)

Return  =>  -0.5199087228837029
Number of trades  =>  32
Winning trades (%)  =>  13.185155047860912
Losing trades (%)  =>  -18.041806898599972
Max drawdown  =>  -0.898704084937917
Sharpe ratio  =>  0.08166485971239659


Bitcoin run at a loss of almost 52% of the initial cash. This strategy does not seem to work for Bitcoin. Let's move on to Ethereum.

## Ethereum

In [134]:
eth = rsi_backtest("ETH-USD")
plot_pf(eth)

Return  =>  -0.8700508727155261
Number of trades  =>  19
Winning trades (%)  =>  14.95548416992863
Losing trades (%)  =>  -27.077189061540352
Max drawdown  =>  -0.9428024607794175
Sharpe ratio  =>  -0.3259822862101921


Looks like it's a lot worse for Ethereum with 87% loss of initial deposit. We can go on and on like this with the other coins, but we would be violating the DRY rule. Instead, Let's collect the metrics of these coins into a dataframe for better view and comparisons.  

In [138]:
# the coins name and their symbol on yahoo finance
coin_dict = {"Bitcoin": "BTC-USD", "Ethereum": "ETH-USD",
             "Chain Native Token": "BNB-USD", "Ripple": "XRP-USD",
             "Enegra": "EGX-USD", "Cardano": "ADA-USD",
             "Solana": "SOL-USD", "DogeCoin": "DOGE-USD",
             "Polygon": "MATIC-USD", "Dai": "DAI-USD"}


coins_metrics = []
for k,v in coin_dict.items():
    rsi = rsi_backtest(v)
    coins_metrics.append({k: rsi})
coins_metrics

[{'Bitcoin': {'Return': -0.5190582741085286,
   'Number of trades': 32,
   'Winning trades (%)': 13.185155047860912,
   'Losing trades (%)': -18.041806898599972,
   'Max drawdown': -0.898704084937917,
   'Sharpe ratio': 0.08209546490867871}},
 {'Ethereum': {'Return': -0.8698658824551795,
   'Number of trades': 19,
   'Winning trades (%)': 14.95548416992863,
   'Losing trades (%)': -27.077189061540352,
   'Max drawdown': -0.9428024607794175,
   'Sharpe ratio': -0.32553785494415055}},
 {'Chain Native Token': {'Return': -0.5288038643421423,
   'Number of trades': 17,
   'Winning trades (%)': 16.83573374675876,
   'Losing trades (%)': -18.249599906019263,
   'Max drawdown': -0.8690008096813584,
   'Sharpe ratio': 0.0946046711493684}},
 {'Ripple': {'Return': -0.7649491513262638,
   'Number of trades': 21,
   'Winning trades (%)': 18.48909167481389,
   'Losing trades (%)': -23.24655901833133,
   'Max drawdown': -0.8735460451540114,
   'Sharpe ratio': 0.01635367283319278}},
 {'Enegra': {'Retu

This returned a list of dictionary, let's convert. it to a dataframe

In [151]:
df_list = [pd.DataFrame(d).transpose() for d in coins_metrics]
df_list

[         Losing trades (%)  Max drawdown  Number of trades    Return  \
 Bitcoin         -18.041807     -0.898704              32.0 -0.519058   
 
          Sharpe ratio  Winning trades (%)  
 Bitcoin      0.082095           13.185155  ,
           Losing trades (%)  Max drawdown  Number of trades    Return  \
 Ethereum         -27.077189     -0.942802              19.0 -0.869866   
 
           Sharpe ratio  Winning trades (%)  
 Ethereum     -0.325538           14.955484  ,
                     Losing trades (%)  Max drawdown  Number of trades  \
 Chain Native Token           -18.2496     -0.869001              17.0   
 
                       Return  Sharpe ratio  Winning trades (%)  
 Chain Native Token -0.528804      0.094605           16.835734  ,
         Losing trades (%)  Max drawdown  Number of trades    Return  \
 Ripple         -23.246559     -0.873546              21.0 -0.764949   
 
         Sharpe ratio  Winning trades (%)  
 Ripple      0.016354           18.489092  ,


In [152]:
df = pd.concat(df_list)
df

,Losing trades (%),Max drawdown,Number of trades,Return,Sharpe ratio,Winning trades (%)
Bitcoin,-18.041807,-0.898704,32.0,-0.519058,0.082095,13.185155
Ethereum,-27.077189,-0.942802,19.0,-0.869866,-0.325538,14.955484
Chain Native Token,-18.249600,-0.869001,17.0,-0.528804,0.094605,16.835734
Ripple,-23.246559,-0.873546,21.0,-0.764949,0.016354,18.489092
Enegra,-0.447760,-0.563205,5.0,1.184445,1.556841,32.378095
Cardano,-38.362069,-0.989072,16.0,-0.983714,-0.655756,16.185020
Solana,-25.899268,-0.796102,9.0,0.127875,0.467679,22.911425
DogeCoin,-19.834770,-0.887067,22.0,-0.846286,-0.187823,12.637693
Polygon,-24.125555,-0.757272,15.0,6.234477,1.056865,39.721308
Dai,-0.391104,-0.021112,2.0,-0.007815,-0.083613,NaN


The only coins that made positive returns are Polygon, Solana and Enegra, with the sharpe ratio of 1.05, 0.47 and 1.56 repectively. Cardano is the biggest loser with a -98% loss, followed by Ethereum and Dogecoin with -87 and -85 respectively.